# 파이썬으로 이미지파일 다루기

## 디지털 이미지

* 디지털 화면은 수많은 점들로 이루어져있으며, 색상이 가지는 점 하나를 화소(pixel)이라고 한다.<br> 각 pixel은 R,G,B(Red, Green, Blue) 세개의 단일 색의 강도를 조절하여 색상표현

* 비트맵(bitmap)방식 : 한점마다 색상별로 8비트를 사용하여 0~255사이 값으로 해당색의 감도 표시

* 벡터(vector)방식 : 상대적인 점과 선의 위치를 방정식으로써 기록해두었다가, 확대 및 축소에 따라 디지털 화면의 각화소에 어떻게 표현될지 계산 >> 깨짐이 없음(확대, 축소 가능한 글꼴)

* 색을 구성하는 방식 >>컬러스페이스, 스페이스 구성 단일축(R, G, B) >>채널(chanel)

jpeg : 근처의 화소를 묶어 비슷한색을 뭉뚱그리는 방식 >>재압축 반복시 색상지저분(디지털풍화)<br>
PNG : 색상의 손실없이 이미지 압축, 사용색상많아지면 jpeg보다 큰 용량 차지

## Pillow 사용법

* numpy와 결합하여 편리하게 사용가능한 이미지처리 도구

* 이미지는 배열형태의 데이터 >>>32픽셀, 3색상 >> [32, 32, 3]

In [1]:
# 32 X 32 pixel짜리 검은색 화면
import numpy as np
from PIL import Image

data = np.zeros([32, 32, 3], dtype=np.uint8) #256사이의 값으로 데이터(1pixel) 표현가능하므로 uint8
image = Image.fromarray(data, 'RGB')
image.show()


In [2]:
data[:, :] = [255, 0, 0]
image = Image.fromarray(data, 'RGB')
image.show()

#  [[* ][* ][* ]] R (255X255X1)
#  [[* ][* ][* ]] G "
#  [[* ][* ][* ]] B " >>>255X255X3

In [3]:
# 가로 세로 각 128 픽셀짜리 흰색 이미지 만들기
data1 = np.zeros([128,128,3], dtype=np.uint8)
data1[:,:] = [255, 255, 255]
image1 = Image.fromarray(data1, 'RGB')
image1.show()

* 연습용 이미지를 열어 width(가로)와 height(세로)를 출력하고 save를 이용하여 jpg포맷으로 저장하기

In [4]:
from PIL import Image
import os

# 연습용파일 경로
image_path = os.getenv('HOME')+'/aiffel/python_image_proc/pillow_practice.png'

# 이미지 열기 (open이용)
img = Image.open(image_path)
img.show()

# width와 height 출력
print(img.width)
print(img.height)

# JPG 파일 형식으로 저장해보기
new_image_path = os.getenv('HOME')+'/aiffel/python_image_proc/pillow_practice.jpg'
img = img.convert('RGB')
img.save(new_image_path)

620
465


* resize() 이용하여 이미지 크기를 100 * 200으로 변경하여 저장

In [5]:
resized_image = img.resize((100, 200))
resized_image.show()

# save
resized_image_path = os.getenv('HOME')+'/aiffel/python_image_proc/pillow_practice_resized.png'
resized_image.save(resized_image_path)

* crop() 이용하여 눈 부분만 잘라내어 저장하기 (눈부분 box좌표: 300,100,600,400)

In [6]:
box = (300, 100, 600, 400)
region = img.crop(box)
region.show()

cropped_image_path = os.getenv('HOME')+'/aiffel/python_image_proc/pillow_practice_cropped.png'
region.save(cropped_image_path)

## Pillow를 활용한 데이터 전처리

### CIFAR-100 데이터를 받아 개별 이미지 파일로 추출하기

* 실습 파일(CIFAR-100 python version) : 32 * 32화소 이미지 100개 클래스당 600장(train:500, test:100) 총 6만장

In [7]:
import os
import pickle

dir_path = os.getenv("HOME")+'/aiffel/python_image_proc/cifar-100-python'
# os.path.join 경로를 병합하여 새 경로 생성
train_file_path = os.path.join(dir_path, 'train') # dir_path/train?

# with문 나올때 close 자동으로 불러줌
# 대부분 인코딩 형태 (읽기-rb, 쓰기-wb, 한작업엔 한개만 하는게 좋음)
with open(train_file_path, 'rb') as f:
    train = pickle.load(f, encoding='bytes')
    
type(train)

dict

In [8]:
train.keys()

dict_keys([b'filenames', b'batch_label', b'fine_labels', b'coarse_labels', b'data'])

In [9]:
type(train[b'filenames'])

list

In [10]:
train[b'filenames'][0:5]

[b'bos_taurus_s_000507.png',
 b'stegosaurus_s_000125.png',
 b'mcintosh_s_000643.png',
 b'altar_boy_s_001435.png',
 b'cichlid_s_000031.png']

In [11]:
train[b'data'][0:5]

array([[255, 255, 255, ...,  10,  59,  79],
       [255, 253, 253, ..., 253, 253, 255],
       [250, 248, 247, ..., 194, 207, 228],
       [124, 131, 135, ..., 232, 236, 231],
       [ 43,  32,  87, ...,  60,  29,  37]], dtype=uint8)

In [12]:
train[b'data'][0].shape

(3072,)

* 3072 = 3채널 * 1024(32*32)<br>
* numpy배열을 reshape하면 이미지파일 원본 복구 가능

In [15]:
# reshape를 앞에서채우는방식이 아닌 원하는 형태로
image_data = train[b'data'][0].reshape([32,32,3], order='F')
# Pillow를 사용하여 Numpy 배열을 Image객체로 만들어서
image = Image.fromarray(image_data)
image.show()

* 사이즈가 작지만 정상이미지, but X축, Y축 뒤집어져 나옴

In [14]:
# 축 바꿔주기
# np.swapaxes(0,1)
image_data = image_data.swapaxes(0,1)
image.show()

* 이미지파일을 실제파일처럼 만들어주는 반복작업을 tqdm이용하여 시각화 가능

In [17]:
import os
import pickle
from PIL import Image
import numpy
from tqdm import tqdm

dir_path = os.getenv('HOME')+'/aiffel/python_image_proc/cifar-100-python'
train_file_path = os.path.join(dir_path, 'train')

# image를 저장할 cifar-100-python의 하위디렉토리(images)를 생성합니다.
# /cifar-100-python/images
images_dir_path = os.path.join(dir_path, 'images')
if not os.path.exists(images_dir_path):
    os.mkdir(images_dir_path)
    
# 32*32 이미지파일 50000개를 생성
with open(train_file_path, 'rb') a f:
    train = pickle.load(f, encoding='bytes')
    for i in tqdm(range(len(train[b'filenames'])))